In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [2]:
from pathlib import Path
from src.data import data_handle
from src.features.features import FraudFeatureEgineering
from src.models.fraud_mode import *
from src.utils import split_train_test_val_set

In [3]:
ROOT_DIR = Path.cwd().parents[0]
data_path = os.path.join(ROOT_DIR, 'data/raw/credit_card.parquet')

In [4]:
data = data_handle.load_data(data_path)

In [5]:
train_set, test_set, val_set = split_train_test_val_set(df=data, 
                                                        train_size=0.8,
                                                        val_size=0.5)

In [6]:
print(train_set.shape, test_set.shape, val_set.shape)

(227984, 35) (28457, 35) (28366, 35)


In [7]:
data_processor = FraudFeatureEgineering(data)

In [8]:
process_data = data_processor.get_fraud_features()

In [9]:
data_process_path = os.path.join(ROOT_DIR, 'data/processed/process_data.parquet')
data_handle.save_data(data_process_path, process_data)

In [10]:
from src.visualization import plots

In [11]:
plots.get_eda_html(file_name='raw_data_eda.html', data=data)

                                             |          | [  0%]   00:00 -> (? left)

Report /Users/kimalejandromoratrujillo/Documents/code/challenge_conekta/fraud_model/reports/raw_data_eda.html was generated.


In [12]:
plots.get_eda_html(file_name='process_data_eda.html', data=process_data)

                                             |          | [  0%]   00:00 -> (? left)

Report /Users/kimalejandromoratrujillo/Documents/code/challenge_conekta/fraud_model/reports/process_data_eda.html was generated.


In [11]:
train_set, test_set, val_set = split_train_test_val_set(df=process_data,
                                                        train_size=0.8,
                                                        val_size=0.5)

In [16]:
train_path = os.path.join(ROOT_DIR, 'data/processed/train_set.parquet')
test_path = os.path.join(ROOT_DIR, 'data/processed/test_set.parquet')
val_path = os.path.join(ROOT_DIR, 'data/processed/validation_set.parquet')

data_handle.save_data(train_path, train_set)
data_handle.save_data(test_path, test_set)
data_handle.save_data(val_path, val_set)

In [12]:
create_tuned_model(train_set=train_set,
                   model_type='lr')

,Description,Value
0,Session id,123
1,Target,Class
2,Target type,Binary
3,Original data shape,"(227984, 47)"
4,Transformed data shape,"(386988, 47)"
5,Transformed train set shape,"(318592, 47)"
6,Transformed test set shape,"(68396, 47)"
7,Numeric features,38
8,Categorical features,2
9,Rows with missing values,73.5%


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9984,0.9946,0.9655,0.5283,0.6829,0.6822,0.7136
1,0.9985,0.9981,0.9655,0.5490,0.7000,0.6993,0.7275
2,0.9985,0.9913,0.8966,0.5532,0.6842,0.6835,0.7036
3,0.9985,0.9648,0.8966,0.5532,0.6842,0.6835,0.7036
4,0.9980,0.9774,0.9655,0.4746,0.6364,0.6355,0.6762
5,0.9984,0.9866,0.8966,0.5306,0.6667,0.6659,0.6890
6,0.9989,0.9971,0.9000,0.6429,0.7500,0.7495,0.7601
7,0.9982,0.9925,0.9333,0.5185,0.6667,0.6659,0.6950
8,0.9976,0.9890,0.9655,0.4242,0.5895,0.5884,0.6392


Transformation Pipeline and Model Successfully Saved


In [13]:
create_tuned_model(train_set=train_set,
                   model_type='catboost')

,Description,Value
0,Session id,123
1,Target,Class
2,Target type,Binary
3,Original data shape,"(227984, 47)"
4,Transformed data shape,"(386988, 47)"
5,Transformed train set shape,"(318592, 47)"
6,Transformed test set shape,"(68396, 47)"
7,Numeric features,38
8,Categorical features,2
9,Rows with missing values,73.5%


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9998,0.9998,0.9655,0.9333,0.9492,0.9491,0.9492
1,0.9999,1.0000,0.9655,1.0000,0.9825,0.9824,0.9826
2,0.9997,0.9999,0.9310,0.9000,0.9153,0.9151,0.9152
3,0.9996,0.9773,0.9310,0.8710,0.9000,0.8998,0.9003
4,0.9996,0.9935,0.8966,0.8667,0.8814,0.8811,0.8813
5,0.9996,0.9829,0.8966,0.8966,0.8966,0.8964,0.8964
6,0.9996,0.9872,0.8667,0.9286,0.8966,0.8964,0.8969
7,0.9998,0.9969,0.9667,0.9355,0.9508,0.9507,0.9509
8,0.9998,1.0000,1.0000,0.9062,0.9508,0.9507,0.9519


Transformation Pipeline and Model Successfully Saved


In [14]:
create_tuned_model(train_set=train_set,
                   model_type='lightgbm')

,Description,Value
0,Session id,123
1,Target,Class
2,Target type,Binary
3,Original data shape,"(227984, 47)"
4,Transformed data shape,"(386988, 47)"
5,Transformed train set shape,"(318592, 47)"
6,Transformed test set shape,"(68396, 47)"
7,Numeric features,38
8,Categorical features,2
9,Rows with missing values,73.5%


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9997,0.9976,0.8966,0.9286,0.9123,0.9121,0.9123
1,0.9997,0.9999,0.9310,0.9310,0.9310,0.9309,0.9309
2,0.9996,0.9996,0.8966,0.8966,0.8966,0.8964,0.8964
3,0.9997,0.9961,0.9310,0.9000,0.9153,0.9151,0.9152
4,0.9997,0.9781,0.8966,0.9286,0.9123,0.9121,0.9123
5,0.9994,0.9766,0.8276,0.8571,0.8421,0.8418,0.8420
6,0.9997,0.9918,0.8333,1.0000,0.9091,0.9089,0.9127
7,0.9997,0.9907,0.9667,0.9062,0.9355,0.9354,0.9358
8,0.9998,1.0000,0.9310,0.9643,0.9474,0.9473,0.9474


Transformation Pipeline and Model Successfully Saved


In [15]:
create_tuned_model(train_set=train_set,
                   model_type='xgboost')

,Description,Value
0,Session id,123
1,Target,Class
2,Target type,Binary
3,Original data shape,"(227984, 47)"
4,Transformed data shape,"(386988, 47)"
5,Transformed train set shape,"(318592, 47)"
6,Transformed test set shape,"(68396, 47)"
7,Numeric features,38
8,Categorical features,2
9,Rows with missing values,73.5%


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.9997,0.9997,0.8966,0.9286,0.9123,0.9121,0.9123
1,0.9999,0.9999,0.9310,1.0000,0.9643,0.9642,0.9648
2,0.9996,0.9996,0.8966,0.8966,0.8966,0.8964,0.8964
3,0.9997,0.9987,0.9655,0.9032,0.9333,0.9332,0.9337
4,0.9996,0.9852,0.8966,0.8966,0.8966,0.8964,0.8964
5,0.9997,0.9732,0.8621,0.9615,0.9091,0.9089,0.9103
6,0.9997,0.9791,0.8333,1.0000,0.9091,0.9089,0.9127
7,0.9999,0.9967,0.9667,0.9667,0.9667,0.9666,0.9666
8,0.9997,1.0000,0.8966,0.9630,0.9286,0.9284,0.9290


Transformation Pipeline and Model Successfully Saved


In [18]:
lr_model = load_model('lr')
xgb_model = load_model('xgboost')
cat_model = load_model('catboost')
lgbm_model = load_model('lightgbm')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


In [19]:
lr_model

Pipeline(memory=FastMemory(location=/var/folders/28/30zl7z411bz8mp5lj62f7qnm0000gn/T/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['V1', 'V2', 'V3', 'V4', 'V5', 'V6',
                                             'V7', 'V8', 'V9', 'V10', 'V11',
                                             'V12', 'V13', 'V14', 'V15', 'V16',
                                             'V17', 'V18', 'V19', 'V20', 'V21',
                                             'V22', 'V23', 'V24', 'V25', 'V26',
                                             'V27', 'V28', 'Amount',
                                             'time_between_transa...
                 TransformerWrapper(exclude=None, include=None,
                                    transformer=StandardScaler(copy=True,
                                                               with_mean=True,
                                                               with_std=True))),
                ('actual_estimator',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=1000,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=123,
                                    solver='lbfgs', tol=0.0001, verbose=0,
                                    warm_start=False))],
         verbose=False)

In [33]:
process_data['score'] = xgb_model.predict_proba(process_data.drop(['Class', 'timestamp', 'credit_card_number'], axis=1))[:,1]